# Lecture 7 (Week 10) Exercise / Homework #3

For this assignment, you will learn about two new libraries, pyttsx3 and speech_recognition, and use voice recognition to obtain weather info <br />

**pyttsx3** - https://pypi.org/project/pyttsx3/ <br />
**speech_recognition** - https://pypi.org/project/SpeechRecognition/ <br />

**pyAudio** - PyAudio‑0.2.11‑cp38‑cp38‑win_amd64.whl <br />
Use following stackoverflow post to troubleshoot if needed 
https://stackoverflow.com/questions/61290821/error-command-errored-out-with-exit-status-1-while-installing-pyaudio

First, try to import the two libraries to see if can import the modules, if not, you need to do pip install to have them installed through terminal (MAC) or command line (MS)

In [1]:
pip install yahoo_fin

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyttsx3
import speech_recognition as sr
import smtplib
from os.path import basename
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
import yfinance as yf
from pandas_datareader import data as pdr
import pandas as pd
from yahoo_fin import stock_info as si

Try out the test script from ptytsx3 website, see if it runs, did you hear the computer speak?

In [3]:
engine = pyttsx3.init()
spell="Wingardium Leviosa!"
engine.say(spell)
engine.runAndWait()

Based on website description, see if you can make engine speak in female voice (**hint:** try voices 1, and 10)

In [4]:
voices = engine.getProperty('voices') 
engine.setProperty('voice', voices[10].id)  
engine.say(spell)
engine.runAndWait()

Check your microphone

In [5]:
for index, name in enumerate(sr.Microphone.list_microphone_names()):
    print(f'{index}, {name}')

0, Built-in Microphone
1, Built-in Output
2, Multi-Output Device


Now let's try listening, try to speak to it, does it provide correct return?
**Note:** only talk to the computer after "listening" is printed

In [6]:
##Make sure you install pyaudio first!!

listener = sr.Recognizer()

try:
    with sr.Microphone(device_index = 0) as source:
        print('listening...')
        voice = listener.listen(source, timeout=4) ##timeout after 2 seconds
        command = listener.recognize_google(voice, language = 'us-eng')
        print(command)
except:
    print("Nothing is Captured")

listening...
Nothing is Captured


Now you can get Python to both listen and talk, let's leverage what we learned in class and have computer tell us weather of a city
for the first test, let's only worry about the "happy path", don't worry about error for now
1. Computer initiate by asking what city you are looking for?
2. User provide a city
3. Computer ask for state abbreviation
4. User provide state abrreviation 
5. System reads out weather description, temperature and feels like (all part of json readout) 

In [7]:
from urllib.request import urlopen  ##import a function
import json

1. Computer initiate by asking what city you are looking for?

In [8]:
def record():
    listener = sr.Recognizer()
    captured_text = ''
    try:
        with sr.Microphone(device_index = 0) as source:
            print('listening...')
            listener.adjust_for_ambient_noise(source, duration=0.5)
            voice = listener.listen(source, timeout=2) ##timeout after 6 seconds
            captured_text = listener.recognize_google(voice, language = 'us-eng')
            print(f"YOU:{captured_text}\n")

    except:
        print("Nothing is Captured")
    return captured_text


In [9]:
def ai_speaks(prompt):
    print(f"AI:{prompt}\n")

    engine = pyttsx3.init()
    engine.say(prompt)
    engine.runAndWait()

In [10]:
# ai_speaks(res)

In [11]:
def get_temp(K):
    C = K - 273
    F = 1.8*C + 32
    return F

In [12]:
def get_weather_report(city, state_abv):
    
    # Get weather data dictionary
    api_key = '09f706cb90f6cc9bf11e5e4bc021a030'
    city = '+'.join(city.lower().split())
    state_abv = state_abv.lower()
    my_url = f"http://api.openweathermap.org/data/2.5/weather?q={city},{state_abv},US&appid={api_key}"
    response = urlopen(my_url)
    rawWeatherData = response.read().decode("utf-8") ##read into text
    weatherData = json.loads(rawWeatherData)
    # generate weather report
    description = weatherData['weather'][0]['description']
    readings = weatherData['main']
    vals = []
    i=0
    
    # First four fields are temperature fields
    for field in readings:
        if i < 4:
            val = round(get_temp(readings[field]))
        else:
            val = readings[field]
        vals.append(val)
        i += 1
        
    description_readout = "We expect {}.\n".format(description)
    temp_readout = """We have the following four temperatures in degrees fahrenheit:
    actual: {},
    feels like: {}.
    The low temperature for today is {} and the high temperature is {}.
    The pressure is {} hPa.
    The humidity is {} percent.
    """.format(*vals)
    
    wind_speed = weatherData['wind']['speed']
    wind_readout = f"We will have {round(2.23694*wind_speed)} miles per hour winds"
    
    full_report = description_readout + temp_readout + wind_readout
    return full_report
    
    
    
    
    

In [13]:
def get_weather():
    city = state_abv = ''
    i = 1
    while(True):
        print(f"Iteration {i}:")
        ai_speaks("What city would you like to know the weather for?")
        city = record()
        ai_speaks(f"Is the city {city} correct? Please answer yes or no.")
        yes_no = record()
        i += 1

        if yes_no == 'yes':
            break
        elif i==4:
            ai_speaks("I'm sorry, I can't understand you even though you tried 3 times \
            Please choose a different language or turn me off and on again")
            return
        print("***************************************************************")
    
    i = 1
    while(True):
        print(f"Iteration {i}:")
        ai_speaks(f"What is the two letter state abbreviation of {city}?")
        state_abv = record()
        ai_speaks(f"Is the state abbreviation {state_abv} correct? Please answer yes or no.")
        yes_no = record()
        i += 1

        if yes_no == 'yes':
            break
        elif i==4:
            ai_speaks("I'm sorry, I can't understand you even though you tried 3 times \
            Please choose a different language or turn me off and on again")
            return
        print("***************************************************************")
    
    report = get_weather_report(city=city, state_abv=state_abv)
    ai_speaks(report)
    
    
        

        
        

In [14]:
get_weather()

Iteration 1:
AI:What city would you like to know the weather for?

listening...
YOU:Highland Park

AI:Is the city Highland Park correct? Please answer yes or no.

listening...
Nothing is Captured
***************************************************************
Iteration 2:
AI:What city would you like to know the weather for?

listening...
YOU:Highland Park

AI:Is the city Highland Park correct? Please answer yes or no.

listening...
Nothing is Captured
***************************************************************
Iteration 3:
AI:What city would you like to know the weather for?

listening...
YOU:Highland Park

AI:Is the city Highland Park correct? Please answer yes or no.

listening...
YOU:yes

Iteration 1:
AI:What is the two letter state abbreviation of Highland Park?

listening...
YOU:NJ

AI:Is the state abbreviation NJ correct? Please answer yes or no.

listening...
Nothing is Captured
***************************************************************
Iteration 2:
AI:What is the two l

In [15]:
get_weather()

Iteration 1:
AI:What city would you like to know the weather for?

listening...
Nothing is Captured
AI:Is the city  correct? Please answer yes or no.

listening...
YOU:no

***************************************************************
Iteration 2:
AI:What city would you like to know the weather for?

listening...
YOU:Isabel Bloom

AI:Is the city Isabel Bloom correct? Please answer yes or no.

listening...
YOU:no

***************************************************************
Iteration 3:
AI:What city would you like to know the weather for?

listening...
YOU:blue blue blue

AI:Is the city blue blue blue correct? Please answer yes or no.

listening...
YOU:no

AI:I'm sorry, I can't understand you even though you tried 3 times             Please choose a different language or turn me off and on again



Were there times the voice didn't capture correctly? Convert it into a function
Please leverage various parameters with speech_recognition to optimize output
In addition, please write some looping code to loop back for potential errors
1. For example, if 'MJ' is captured instead of 'NJ', we should ask user to confirm
2. Timeout errors

Let's leverage what you learned in class this week and design a quick tool

In [16]:
import pandas as pd
import numpy as np

###Symbol column will provide you ticker of all 30 index
ticker = pd.read_html('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average')
ticker[1]

,Company,Exchange,Symbol,Industry,Date added,Notes,Index weighting
0,3M,NYSE,MMM,Conglomerate,1976-08-09,As Minnesota Mining and Manufacturing,3.02%
1,American Express,NYSE,AXP,Financial services,1982-08-30,NaN,3.60%
2,Amgen,NASDAQ,AMGN,Biopharmaceutical,2020-08-31,NaN,4.48%
3,Apple,NASDAQ,AAPL,Information technology,2015-03-19,NaN,3.25%
4,Boeing,NYSE,BA,Aerospace and defense,1987-03-12,NaN,3.96%
5,Caterpillar,NYSE,CAT,Construction and Mining,1991-05-06,NaN,3.74%
6,Chevron,NYSE,CVX,Petroleum industry,2008-02-19,Also 1930-07-18 to 1999-11-01,2.53%
7,Cisco,NASDAQ,CSCO,Information technology,2009-06-08,NaN,1.03%
8,Coca-Cola,NYSE,KO,Drink industry,1987-03-12,Also 1932-05-26 to 1935-11-20,1.15%
9,Disney,NYSE,DIS,Broadcasting and entertainment,1991-05-06,NaN,2.65%


In [17]:
###Pull volume of stock for individual ticker
import requests
url_link = 'https://finance.yahoo.com/quote/MMM/key-statistics?p=MMM'
enhanced_link = requests.get(url_link,headers ={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'})
MMM = pd.read_html(enhanced_link.text)
MMM[0]

,Unnamed: 0,As of Date: 3/30/2022Current,12/31/2021,9/30/2021,6/30/2021,3/31/2021
0,Market Cap (intraday),86.56B,101.58B,101.09B,114.93B,111.69B
1,Enterprise Value,100.10B,114.85B,114.71B,128.86B,126.41B
2,Trailing P/E,15.03,17.57,17.35,20.33,20.83
3,Forward P/E,14.62,17.15,16.29,20.49,19.96
4,PEG Ratio (5 yr expected),2.09,2.16,1.82,2.13,3.25
5,Price/Sales (ttm),2.52,2.95,2.96,3.52,3.49
6,Price/Book (mrq),5.75,6.99,6.96,8.35,8.68
7,Enterprise Value/Revenue,2.83,13.34,12.83,14.40,14.28
8,Enterprise Value/EBITDA,10.42,52.47,48.81,50.91,49.83


Write a program that will send your self an email on stock summary based on last trading day (You may assume the code is only ran after market is closed)
1. Stock Ticker with highest volume
2. Stock Ticker with largest fluctuation (High - Low) / Low
3. Stock Ticker with max gain (Close-Open) / Open
4. Stock Ticket with max loss or min gain (Close-Open) / Open
Once you have your code finalized, have a copy send to rutgersadvancedpython01@outlook.com

In [18]:
resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
table1 = pd.read_html(resp.text)
tickers = list(table1[0]['Symbol'])
tickers

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADM',
 'ADBE',
 'ADP',
 'AAP',
 'AES',
 'AFL',
 'A',
 'AIG',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ANET',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'WRB',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'CHRW',
 'CDNS',
 'CZR',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CERN',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMC

In [21]:
df = pd.DataFrame()

for t in tickers:
    my_ticker = t

    ticker_data = yf.Ticker(my_ticker)

    ticker_df = ticker_data.history(period='1d', start='2022-3-31', end='2022-4-1')
    ticker_df['stock'] = my_ticker
    ticker_df.set_index('stock', inplace = True)

    df = df.append(ticker_df)

- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


In [22]:
df.reset_index(inplace =True)

In [28]:
df['fluctuation'] =(df.High - df.Low) / df.Low
df['gain']=(df.Close-df.Open) / df.Open

fluctuation_stock =df[df.fluctuation ==df.fluctuation.max()].stock.item()
max_stock = df[df.gain == df.gain.max()].stock.item()
min_stock = df[df.gain == df.gain.min()].stock.item()

max_stock

'MKC'

In [35]:
text = f"""The stock which showed the highest fluctuation is: {fluctuation_stock}.
        The stock which showed the highest gain is: {max_stock}
        Finally, the stock which showed the least gain is: {min_stock}""" 

In [36]:
print(text)

The stock which showed the highest fluctuation is: POOL.
        The stock which showed the highest gain is: MKC
        Finally, the stock which showed the least gain is: ETSY


In [79]:
def send_email(text):
    msg = MIMEMultipart()
    msg['Subject'] = "Dow Jones stocks"
    
    msg['From'] = '15jorada@gmail.com'

    msg['To'] = '15jorada@gmail.com'
    
    html = """\
    <html>
      <head></head>
        <body>
           <p>
           """ + text + """\
           </p>   
           <img src="cid:image1" alt="Logo"><br>
        </body>
    </html>
    """
    # Get MIME types
    mime_types = MIMEText(html, 'html')
    msg.attach(mime_types)
    
    s = smtplib.SMTP("smtp.gmail.com", 587)
    s.ehlo()
    s.starttls()
    s.login('15jorada@gmail.com', 'YOURPASSWORD')
    s.sendmail(email_to[0], 'nakul.datla@gmail.com', msg.as_string())

In [80]:
def send_email(text):
    msg = MIMEMultipart()
    msg['Subject'] = "Dow Jones stocks"
    
    msg['From'] = '15jorada@gmail.com'

    msg['To'] = '15jorada@gmail.com'
    
    html = """\
    <html>
      <head></head>
        <body>
           <p>
           """ + text + """\
           </p>   
           <img src="cid:image1" alt="Logo"><br>
        </body>
    </html>
    """
    # Get MIME types
    mime_types = MIMEText(html, 'html')
    msg.attach(mime_types)
    
    s = smtplib.SMTP("smtp.gmail.com", 587)
    s.ehlo()
    s.starttls()
    s.login('15jorada@gmail.com', 'YOURPASSWORD')
    s.sendmail(email_to[0], 'nakul.datla@gmail.com', msg.as_string())send_email(text)